# Agent Frameworks: LangChain and AutoGen

This notebook provides comprehensive implementations of AI agent frameworks using LangChain and AutoGen, covering from basic agents to advanced multi-agent systems.

## 1. Setup and Imports

In [ ]:
# Core imports
import os
import json
import time
import asyncio
from typing import List, Dict, Any, Optional, Callable
from dataclasses import dataclass, field
from abc import ABC, abstractmethod
from enum import Enum
import logging
from datetime import datetime

# LangChain imports
from langchain.agents import AgentType, Tool, AgentExecutor, initialize_agent
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain.chains import LLMChain, SimpleSequentialChain, TransformChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.tools import BaseTool
from langchain.callbacks.base import BaseCallbackHandler

# AutoGen imports
try:
    import autogen
    from autogen import Agent, ConversableAgent, GroupChat, GroupChatManager
    from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
    from autogen.agentchat.contrib.gpt_assistant_agent import GPTAssistantAgent
except ImportError:
    print("AutoGen not available. Install with: pip install pyautogen")

# LLM imports
try:
    from langchain_openai import ChatOpenAI
except ImportError:
    from langchain.chat_models import ChatOpenAI

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Environment setup
os.environ["OPENAI_API_KEY"] = "your-openai-api-key"  # Replace with your API key

# Initialize LLM
llm = ChatOpenAI(
    model_name="gpt-4",
    temperature=0.7,
    max_tokens=2000
)

## 2. LangChain Agent Framework

In [ ]:
class CustomTool(BaseTool):
    """Custom tool for LangChain agents"""
    name: str = "custom_tool"
    description: str = "A custom tool that can perform various operations"
    
    def __init__(self, func: Callable, name: str, description: str):
        super().__init__()
        self.func = func
        self.name = name
        self.description = description
    
    def _run(self, query: str) -> str:
        """Run the tool"""
        try:
            return self.func(query)
        except Exception as e:
            return f"Error: {str(e)}"

# Custom tools definitions
def calculate_expression(expression: str) -> str:
    """Calculate mathematical expressions"""
    try:
        result = eval(expression)
        return f"The result of {expression} is {result}"
    except:
        return "Invalid mathematical expression"

def search_web(query: str) -> str:
    """Simulate web search"""
    # This is a mock implementation
    search_results = {
        "weather": "The weather today is sunny with 25°C",
        "news": "Latest AI breakthroughs in reinforcement learning",
        "sports": "Local team won the championship game"
        "technology": "New AI models show promising results"
    }
    
    for key, value in search_results.items():
        if key in query.lower():
            return value
    
    return f"No results found for '{query}'. Try searching for weather, news, sports, or technology."

def get_time(query: str) -> str:
    """Get current time"""
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return f"Current time is: {current_time}"

def save_to_file(content: str, filename: str) -> str:
    """Save content to file"""
    try:
        with open(filename, 'w') as f:
            f.write(content)
        return f"Successfully saved content to {filename}"
    except Exception as e:
        return f"Error saving file: {str(e)}"

# Create tools
calculator_tool = CustomTool(
    calculate_expression,
    name="calculator",
    description="Useful for calculating mathematical expressions. Input should be a valid math expression like '2 + 2 * 3'"
)

search_tool = CustomTool(
    search_web,
    name="search",
    description="Useful for searching the web for current information. Input should be a search query."
)

time_tool = CustomTool(
    get_time,
    name="get_time",
    description="Useful for getting the current time and date."
)

file_tool = CustomTool(
    lambda x: save_to_file(x, "output.txt"),
    name="save_file",
    description="Useful for saving content to a file named 'output.txt'."
)

# Tool collection
tools = [calculator_tool, search_tool, time_tool, file_tool]

class LangChainAgentFramework:
    """Comprehensive LangChain agent framework"""
    
    def __init__(self, llm, tools: List[Tool] = None):
        self.llm = llm
        self.tools = tools or []
        self.agents = {}
        self.agent_configs = {}
        
    def create_react_agent(self, name: str, system_prompt: str = None, memory: bool = True):
        """Create a ReAct agent"""
        
        if system_prompt is None:
            system_prompt = """You are a helpful AI assistant with access to various tools. 
            Use the appropriate tools to answer questions and complete tasks.
            Always think step by step and explain your reasoning."""
        
        # Set up memory if requested
        memory_obj = ConversationBufferMemory(memory_key="chat_history", return_messages=True) if memory else None
        
        # Create agent
        agent = initialize_agent(
            tools=self.tools,
            llm=self.llm,
            agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            memory=memory_obj,
            agent_kwargs={
                "system_message": system_prompt,
                "prefix": system_prompt
            }
        )
        
        self.agents[name] = agent
        self.agent_configs[name] = {
            "type": "react",
            "system_prompt": system_prompt,
            "memory": memory
        }
        
        return agent
    
    def create_conversational_agent(self, name: str, system_prompt: str = None):
        """Create a conversational agent"""
        
        if system_prompt is None:
            system_prompt = """You are a friendly and helpful conversational AI assistant.
            Engage in natural conversation and help users with their questions.
            Be conversational and engaging while providing accurate information."""
        
        # Create prompt template
        prompt = ChatPromptTemplate.from_messages([
            SystemMessage(content=system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            HumanMessage(content="{input}")
        ])
        
        # Create memory
        memory = ConversationBufferMemory(return_messages=True)
        
        # Create chain
        chain = LLMChain(
            llm=self.llm,
            prompt=prompt,
            memory=memory,
            verbose=True
        )
        
        self.agents[name] = chain
        self.agent_configs[name] = {
            "type": "conversational",
            "system_prompt": system_prompt,
            "memory": True
        }
        
        return chain
    
    def create_planning_agent(self, name: str, system_prompt: str = None):
        """Create a planning and task decomposition agent"""
        
        if system_prompt is None:
            system_prompt = """You are an expert planner and task decomposition agent.
            When given a complex task, break it down into smaller, manageable steps.
            For each step, explain what needs to be done and why it's important.
            Consider dependencies between steps and suggest an optimal order."""
        
        # Create prompt template for planning
        planning_prompt = PromptTemplate(
            input_variables=["task"],
            template="""{system_prompt}

Task: {task}

Please break this task down into clear steps:

1. [Step 1] - [Reasoning]
2. [Step 2] - [Reasoning]
3. [Step 3] - [Reasoning]

Consider dependencies and provide a logical sequence.""".format(
                system_prompt=system_prompt, task="{task}"
            )
        )
        
        # Create planning chain
        planning_chain = LLMChain(
            llm=self.llm,
            prompt=planning_prompt,
            verbose=True
        )
        
        self.agents[name] = planning_chain
        self.agent_configs[name] = {
            "type": "planning",
            "system_prompt": system_prompt,
            "memory": False
        }
        
        return planning_chain
    
    def create_multi_tool_agent(self, name: str, specific_tools: List[Tool] = None):
        """Create an agent with specific tools"""
        
        tools_to_use = specific_tools or self.tools
        
        system_prompt = """You are a specialized tool-using agent.
        You have access to specific tools to help you complete tasks.
        Use the tools appropriately and explain your actions.
        Always verify the results and provide clear explanations."""
        
        agent = initialize_agent(
            tools=tools_to_use,
            llm=self.llm,
            agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True),
            agent_kwargs={"system_message": system_prompt}
        )
        
        self.agents[name] = agent
        self.agent_configs[name] = {
            "type": "multi_tool",
            "tools": [tool.name for tool in tools_to_use],
            "system_prompt": system_prompt,
            "memory": True
        }
        
        return agent
    
    def run_agent(self, agent_name: str, input_text: str):
        """Run a specific agent"""
        if agent_name not in self.agents:
            raise ValueError(f"Agent '{agent_name}' not found")
        
        agent = self.agents[agent_name]
        config = self.agent_configs[agent_name]
        
        logger.info(f"Running agent '{agent_name}' with input: {input_text}")
        
        try:
            if config["type"] in ["react", "multi_tool"]:
                result = agent.run(input_text)
            elif config["type"] == "conversational":
                result = agent.predict(input=input_text)
            elif config["type"] == "planning":
                result = agent.run(task=input_text)
            else:
                result = agent(input_text)
            
            return result
            
        except Exception as e:
            logger.error(f"Error running agent '{agent_name}': {str(e)}")
            return f"Error: {str(e)}"
    
    def list_agents(self):
        """List all available agents"""
        return list(self.agents.keys())
    
    def get_agent_info(self, agent_name: str):
        """Get information about a specific agent"""
        if agent_name in self.agent_configs:
            return self.agent_configs[agent_name]
        return None

## 3. AutoGen Agent Framework

In [ ]:
class AutoGenFramework:
    """Comprehensive AutoGen agent framework"""
    
    def __init__(self, config_list: List[Dict[str, str]] = None):
        """Initialize AutoGen framework"""
        self.agents = {}
        self.group_chats = {}
        
        # Default configuration
        if config_list is None:
            config_list = [{
                "model": "gpt-4",
                "api_key": os.environ.get("OPENAI_API_KEY"),
            }]
        
        self.config_list = config_list
    
    def create_assistant_agent(self, name: str, system_message: str, **kwargs):
        """Create an assistant agent"""
        
        default_config = {
            "name": name,
            "system_message": system_message,
            "llm_config": {"config_list": self.config_list},
            "human_input_mode": "NEVER",
        }
        
        # Update with provided kwargs
        default_config.update(kwargs)
        
        agent = ConversableAgent(**default_config)
        self.agents[name] = agent
        
        return agent
    
    def create_user_proxy(self, name: str, **kwargs):
        """Create a user proxy agent"""
        
        default_config = {
            "name": name,
            "human_input_mode": "NEVER",
            "max_consecutive_auto_reply": 1,
            "code_execution_config": False,
        }
        
        default_config.update(kwargs)
        
        agent = ConversableAgent(**default_config)
        self.agents[name] = agent
        
        return agent
    
    def create_coding_agent(self, name: str, system_message: str = None, **kwargs):
        """Create a coding assistant agent"""
        
        if system_message is None:
            system_message = """You are a helpful AI assistant that can write and execute code.
            When asked to solve a problem, write the code to solve it and execute it.
            Always explain what the code does and provide the results."""
        
        default_config = {
            "name": name,
            "system_message": system_message,
            "llm_config": {"config_list": self.config_list},
            "human_input_mode": "NEVER",
            "code_execution_config": {
                "work_dir": "coding_workspace",
                "use_docker": False,
            }
        }
        
        default_config.update(kwargs)
        
        agent = ConversableAgent(**default_config)
        self.agents[name] = agent
        
        return agent
    
    def create_group_chat(self, name: str, agent_names: List[str], **kwargs):
        """Create a group chat with multiple agents"""
        
        # Get agents
        agents = [self.agents[name] for name in agent_names if name in self.agents]
        
        if not agents:
            raise ValueError("No valid agents provided for group chat")
        
        # Create group chat
        default_config = {
            "agents": agents,
            "messages": [],
            "max_round": 50,
        }
        
        default_config.update(kwargs)
        
        group_chat = GroupChat(**default_config)
        
        # Create manager
        manager_config = kwargs.get("manager_config", {})
        manager = GroupChatManager(groupchat=group_chat, **manager_config)
        
        self.group_chats[name] = {
            "group_chat": group_chat,
            "manager": manager,
            "agent_names": agent_names
        }
        
        return manager
    
    def initiate_chat(self, agent_name: str, recipient_name: str, message: str, **kwargs):
        """Initiate chat between two agents"""
        
        if agent_name not in self.agents:
            raise ValueError(f"Agent '{agent_name}' not found")
        
        if recipient_name not in self.agents:
            raise ValueError(f"Agent '{recipient_name}' not found")
        
        agent = self.agents[agent_name]
        recipient = self.agents[recipient_name]
        
        result = agent.initiate_chat(recipient, message=message, **kwargs)
        return result
    
    def run_group_chat(self, group_name: str, initiator_name: str, message: str):
        """Run a group chat"""
        
        if group_name not in self.group_chats:
            raise ValueError(f"Group chat '{group_name}' not found")
        
        group_info = self.group_chats[group_name]
        
        if initiator_name not in group_info["agent_names"]:
            raise ValueError(f"Initiator '{initiator_name}' not in group chat")
        
        initiator = self.agents[initiator_name]
        
        result = initiator.initiate_chat(
            group_info["manager"],
            message=message
        )
        
        return result
    
    def list_agents(self):
        """List all available agents"""
        return list(self.agents.keys())
    
    def list_group_chats(self):
        """List all group chats"""
        return list(self.group_chats.keys())

# Example AutoGen setup
def setup_autogen_example():
    """Set up example AutoGen agents and group chat"""
    
    # Initialize framework
    autogen_framework = AutoGenFramework()
    
    # Create agents
    planner = autogen_framework.create_assistant_agent(
        "planner",
        system_message="""You are a project planning expert. 
        Break down complex projects into manageable tasks and create detailed plans.
        Consider timelines, resources, and dependencies."""
    )
    
    coder = autogen_framework.create_coding_agent(
        "coder",
        system_message="""You are an expert programmer. 
        Write clean, efficient code and explain your solutions.
        Test your code and ensure it works correctly."""
    )
    
    reviewer = autogen_framework.create_assistant_agent(
        "reviewer",
        system_message="""You are a code reviewer. 
        Review code for quality, efficiency, and best practices.
        Provide constructive feedback and suggest improvements."""
    )
    
    user_proxy = autogen_framework.create_user_proxy(
        "user_proxy",
        max_consecutive_auto_reply=0
    )
    
    # Create group chat
    project_team = autogen_framework.create_group_chat(
        "project_team",
        ["planner", "coder", "reviewer"],
        max_round=20
    )
    
    return autogen_framework

## 4. Advanced Multi-Agent Systems

In [ ]:
class MultiAgentSystem:
    """Advanced multi-agent system with coordination and communication"""
    
    def __init__(self, langchain_framework, autogen_framework):
        self.langchain = langchain_framework
        self.autogen = autogen_framework
        self.agent_network = {}
        self.workflow_engine = WorkflowEngine()
        
    def create_specialized_agents(self):
        """Create specialized agents for different tasks"""
        
        # Research agent
        research_agent = self.langchain.create_react_agent(
            "researcher",
            system_prompt="""You are a research specialist with access to search tools.
            Conduct thorough research on topics and provide comprehensive summaries.
            Always cite sources and verify information."""
        )
        
        # Analysis agent
        analysis_agent = self.langchain.create_conversational_agent(
            "analyst",
            system_prompt="""You are a data analysis expert.
            Analyze information, identify patterns, and provide insights.
            Use logical reasoning and evidence-based conclusions."""
        )
        
        # Strategy agent
        strategy_agent = self.langchain.create_planning_agent(
            "strategist",
            system_prompt="""You are a strategic planning expert.
            Develop comprehensive strategies and action plans.
            Consider long-term implications and multiple scenarios."""
        )
        
        # Implementation agent
        implementation_agent = self.langchain.create_multi_tool_agent(
            "implementer",
            specific_tools=[calculator_tool, file_tool]
        )
        
        return {
            "researcher": research_agent,
            "analyst": analysis_agent,
            "strategist": strategy_agent,
            "implementer": implementation_agent
        }
    
    def create_agent_workflow(self, workflow_name: str, agents: Dict[str, Any], workflow_config: Dict):
        """Create a workflow with multiple agents"""
        
        workflow = {
            "name": workflow_name,
            "agents": agents,
            "steps": workflow_config.get("steps", []),
            "dependencies": workflow_config.get("dependencies", {}),
            "output_format": workflow_config.get("output_format", "summary")
        }
        
        self.agent_network[workflow_name] = workflow
        return workflow
    
    def execute_workflow(self, workflow_name: str, input_data: Dict):
        """Execute a multi-agent workflow"""
        
        if workflow_name not in self.agent_network:
            raise ValueError(f"Workflow '{workflow_name}' not found")
        
        workflow = self.agent_network[workflow_name]
        agents = workflow["agents"]
        steps = workflow["steps"]
        
        results = {}
        context = {"input": input_data}
        
        for step in steps:
            agent_name = step["agent"]
            task = step["task"]
            
            if agent_name not in agents:
                logger.error(f"Agent '{agent_name}' not found in workflow")
                continue
            
            # Prepare input for agent
            agent_input = self._prepare_agent_input(task, context)
            
            # Execute agent
            logger.info(f"Executing agent '{agent_name}' for task: {task}")
            result = self.langchain.run_agent(agent_name, agent_input)
            
            # Store result
            results[step["output_key"]] = result
            context[step["output_key"]] = result
            
            logger.info(f"Agent '{agent_name}' completed task")
        
        # Format final output
        final_output = self._format_workflow_output(results, workflow["output_format"])
        
        return final_output
    
    def _prepare_agent_input(self, task: str, context: Dict) -> str:
        """Prepare input for agent execution"""
        
        # Replace placeholders in task with context values
        for key, value in context.items():
            task = task.replace(f"{{{key}}}", str(value))
        
        return task
    
    def _format_workflow_output(self, results: Dict, output_format: str) -> str:
        """Format workflow output according to specified format"""
        
        if output_format == "summary":
            summary_parts = []
            for key, value in results.items():
                summary_parts.append(f"{key}: {value}")
            return "\n\n".join(summary_parts)
        
        elif output_format == "detailed":
            detailed_output = f"Workflow Results:\n\n"
            for key, value in results.items():
                detailed_output += f"{key}:\n{value}\n\n"
            return detailed_output
        
        else:
            return str(results)

class WorkflowEngine:
    """Workflow engine for managing multi-agent workflows"""
    
    def __init__(self):
        self.workflows = {}
        self.execution_history = []
    
    def register_workflow(self, name: str, workflow_config: Dict):
        """Register a workflow"""
        self.workflows[name] = workflow_config
    
    def execute_workflow(self, name: str, input_data: Dict):
        """Execute a registered workflow"""
        
        if name not in self.workflows:
            raise ValueError(f"Workflow '{name}' not registered")
        
        workflow = self.workflows[name]
        
        execution_record = {
            "workflow_name": name,
            "input_data": input_data,
            "start_time": datetime.now(),
            "steps": []
            "status": "running"
        }
        
        try:
            results = self._execute_workflow_steps(workflow, input_data, execution_record)
            execution_record["status"] = "completed"
            execution_record["results"] = results
            execution_record["end_time"] = datetime.now()
            
        except Exception as e:
            execution_record["status"] = "failed"
            execution_record["error"] = str(e)
            execution_record["end_time"] = datetime.now()
            
            logger.error(f"Workflow execution failed: {str(e)}")
            
        self.execution_history.append(execution_record)
        return execution_record
    
    def _execute_workflow_steps(self, workflow: Dict, input_data: Dict, execution_record: Dict):
        """Execute workflow steps"""
        
        results = {}
        context = input_data.copy()
        
        for step in workflow["steps"]:
            step_record = {
                "step_name": step["name"],
                "start_time": datetime.now(),
                "status": "running"
            }
            
            try:
                # Check dependencies
                if self._check_dependencies(step, results):
                    # Execute step
                    step_result = self._execute_step(step, context)
                    results[step["output_key"]] = step_result
                    context[step["output_key"]] = step_result
                    
                    step_record["status"] = "completed"
                    step_record["result"] = step_result
                else:
                    step_record["status"] = "skipped"
                    step_record["reason"] = "Dependencies not met"
                
            except Exception as e:
                step_record["status"] = "failed"
                step_record["error"] = str(e)
                
                logger.error(f"Step '{step['name']}' failed: {str(e)}")
                
                if workflow.get("fail_fast", False):
                    raise e
            
            step_record["end_time"] = datetime.now()
            execution_record["steps"].append(step_record)
        
        return results
    
    def _check_dependencies(self, step: Dict, results: Dict) -> bool:
        """Check if step dependencies are satisfied"""
        
        dependencies = step.get("dependencies", [])
        
        for dep in dependencies:
            if dep not in results:
                return False
        
        return True
    
    def _execute_step(self, step: Dict, context: Dict) -> Any:
        """Execute a single workflow step"""
        
        step_type = step["type"]
        
        if step_type == "agent":
            # This would integrate with the agent framework
            agent_name = step["agent"]
            task = step["task"].format(**context)
            
            # Placeholder for agent execution
            return f"Agent '{agent_name}' executed task: {task}"
        
        elif step_type == "transform":
            # Data transformation step
            transform_func = step["function"]
            input_data = context[step["input"]]
            return transform_func(input_data)
        
        elif step_type == "condition":
            # Conditional step
            condition = step["condition"]
            return eval(condition, {}, context)
        
        else:
            raise ValueError(f"Unknown step type: {step_type}")
    
    def get_execution_history(self, workflow_name: str = None) -> List[Dict]:
        """Get execution history"""
        
        if workflow_name:
            return [record for record in self.execution_history 
                    if record["workflow_name"] == workflow_name]
        
        return self.execution_history

## 5. Agent Memory and Learning

In [ ]:
class AgentMemorySystem:
    """Advanced memory system for AI agents"""
    
    def __init__(self, max_short_term: int = 1000, max_long_term: int = 10000):
        self.short_term_memory = []  # Recent interactions
        self.long_term_memory = []   # Persistent knowledge
        self.episodic_memory = []    # Specific episodes/contexts
        self.semantic_memory = {}    # Facts and concepts
        
        self.max_short_term = max_short_term
        self.max_long_term = max_long_term
        
        # Memory indices
        self.semantic_index = {}
        self.episodic_index = {}
    
    def add_short_term_memory(self, memory: Dict):
        """Add to short-term memory"""
        memory["timestamp"] = datetime.now()
        memory["id"] = len(self.short_term_memory)
        
        self.short_term_memory.append(memory)
        
        # Maintain size limit
        if len(self.short_term_memory) > self.max_short_term:
            # Move oldest to long-term memory
            old_memory = self.short_term_memory.pop(0)
            self.add_long_term_memory(old_memory)
    
    def add_long_term_memory(self, memory: Dict):
        """Add to long-term memory"""
        memory["timestamp"] = datetime.now()
        memory["id"] = len(self.long_term_memory)
        
        self.long_term_memory.append(memory)
        
        # Maintain size limit
        if len(self.long_term_memory) > self.max_long_term:
            self.long_term_memory.pop(0)
    
    def add_episodic_memory(self, episode: Dict):
        """Add episodic memory"""
        episode["timestamp"] = datetime.now()
        episode["id"] = len(self.episodic_memory)
        
        self.episodic_memory.append(episode)
        
        # Index episode
        keywords = self._extract_keywords(episode)
        for keyword in keywords:
            if keyword not in self.episodic_index:
                self.episodic_index[keyword] = []
            self.episodic_index[keyword].append(episode["id"])
    
    def add_semantic_memory(self, concept: str, knowledge: Dict):
        """Add semantic memory (facts and concepts)"""
        knowledge["concept"] = concept
        knowledge["timestamp"] = datetime.now()
        
        self.semantic_memory[concept] = knowledge
        
        # Index semantic memory
        keywords = self._extract_keywords(knowledge)
        for keyword in keywords:
            if keyword not in self.semantic_index:
                self.semantic_index[keyword] = []
            self.semantic_index[keyword].append(concept)
    
    def retrieve_relevant_memories(self, query: str, limit: int = 5) -> List[Dict]:
        """Retrieve memories relevant to query"""
        
        # Extract keywords from query
        query_keywords = self._extract_keywords({"query": query})
        
        relevant_memories = []
        memory_scores = {}
        
        # Search short-term memory
        for memory in self.short_term_memory[-50:]:  # Last 50 items
            score = self._calculate_relevance_score(query_keywords, memory)
            if score > 0.1:
                memory_scores[memory["id"]] = (score, memory, "short_term")
        
        # Search semantic memory
        for keyword in query_keywords:
            if keyword in self.semantic_index:
                for concept in self.semantic_index[keyword]:
                    if concept in self.semantic_memory:
                        memory = self.semantic_memory[concept]
                        score = self._calculate_relevance_score(query_keywords, memory)
                        memory_scores[memory["id"]] = (score, memory, "semantic")
        
        # Search episodic memory
        for keyword in query_keywords:
            if keyword in self.episodic_index:
                for episode_id in self.episodic_index[keyword]:
                    if episode_id < len(self.episodic_memory):
                        memory = self.episodic_memory[episode_id]
                        score = self._calculate_relevance_score(query_keywords, memory)
                        memory_scores[memory["id"]] = (score, memory, "episodic")
        
        # Sort by relevance and return top memories
        sorted_memories = sorted(memory_scores.values(), key=lambda x: x[0], reverse=True)
        
        return [memory[1] for memory in sorted_memories[:limit]]
    
    def _extract_keywords(self, text_data: Dict) -> List[str]:
        """Extract keywords from text data"""
        # Simple keyword extraction
        text = str(text_data).lower()
        
        # Remove common stop words
        stop_words = {"the", "a", "an", "and", "or", "but", "in", "on", "at", "to", "for", "of", "with", "by"}
        
        words = text.split()
        keywords = [word.strip(".,?!;:'\"()") for word in words if len(word) > 3 and word not in stop_words]
        
        return list(set(keywords))
    
    def _calculate_relevance_score(self, query_keywords: List[str], memory: Dict) -> float:
        """Calculate relevance score between query and memory"""
        memory_keywords = self._extract_keywords(memory)
        
        # Simple overlap scoring
        overlap = len(set(query_keywords) & set(memory_keywords))
        total_keywords = len(set(query_keywords) | set(memory_keywords))
        
        if total_keywords == 0:
            return 0.0
        
        return overlap / total_keywords
    
    def consolidate_memories(self):
        """Consolidate short-term memories into long-term memory"""
        
        # Move old short-term memories to long-term
        memories_to_consolidate = self.short_term_memory[:len(self.short_term_memory) // 2]
        
        for memory in memories_to_consolidate:
            self.add_long_term_memory(memory)
        
        # Clear from short-term
        self.short_term_memory = self.short_term_memory[len(memories_to_consolidate):]
    
    def get_memory_stats(self) -> Dict:
        """Get memory system statistics"""
        return {
            "short_term_count": len(self.short_term_memory),
            "long_term_count": len(self.long_term_memory),
            "episodic_count": len(self.episodic_memory),
            "semantic_count": len(self.semantic_memory),
            "semantic_index_size": len(self.semantic_index),
            "episodic_index_size": len(self.episodic_index)
        }

class LearningAgent:
    """Agent with learning capabilities"""
    
    def __init__(self, name: str, memory_system: AgentMemorySystem):
        self.name = name
        self.memory_system = memory_system
        self.learning_rate = 0.1
        self.performance_history = []
        self.adaptation_rules = []
    
    def learn_from_interaction(self, interaction: Dict):
        """Learn from agent interactions"""
        
        # Add to memory
        self.memory_system.add_short_term_memory(interaction)
        
        # Extract learning
        learning = self._extract_learning(interaction)
        
        if learning:
            # Add to semantic memory
            self.memory_system.add_semantic_memory(
                f"learned_{len(self.memory_system.semantic_memory)}",
                {
                    "type": "learned_rule",
                    "rule": learning,
                    "context": interaction.get("context", ""),
                    "confidence": 0.8
                }
            )
            
            # Update adaptation rules
            self.adaptation_rules.append(learning)
            
            logger.info(f"Agent {self.name} learned: {learning}")
    
    def adapt_behavior(self, context: Dict) -> Dict:
        """Adapt behavior based on learning"""
        
        # Retrieve relevant memories
        relevant_memories = self.memory_system.retrieve_relevant_memories(
            str(context), limit=3
        )
        
        # Apply adaptation rules
        adapted_behavior = context.copy()
        
        for memory in relevant_memories:
            if memory.get("type") == "learned_rule":
                rule = memory.get("rule", "")
                # Apply rule (simplified)
                if "more_detailed" in rule:
                    adapted_behavior["detail_level"] = adapted_behavior.get("detail_level", 1) + 1
                elif "faster_response" in rule:
                    adapted_behavior["response_speed"] = "fast"
        
        return adapted_behavior
    
    def _extract_learning(self, interaction: Dict) -> Optional[str]:
        """Extract learning from interaction"""
        
        # Simple learning extraction
        outcome = interaction.get("outcome", "")
        feedback = interaction.get("feedback", "")
        
        if "positive" in feedback.lower() and outcome == "success":
            return "Continue this approach, it was successful"
        elif "negative" in feedback.lower():
            return "Adjust approach, it was not well received"
        elif "more detail" in feedback.lower():
            return "Provide more detailed responses"
        elif "faster" in feedback.lower():
            return "Respond more quickly"
        
        return None
    
    def evaluate_performance(self, metrics: Dict):
        """Evaluate and track performance"""
        
        performance_record = {
            "timestamp": datetime.now(),
            "metrics": metrics,
            "trend": self._calculate_performance_trend(metrics)
        }
        
        self.performance_history.append(performance_record)
        
        # Keep only recent history
        if len(self.performance_history) > 100:
            self.performance_history = self.performance_history[-50:]
        
        return performance_record
    
    def _calculate_performance_trend(self, current_metrics: Dict) -> str:
        """Calculate performance trend"""
        
        if len(self.performance_history) < 5:
            return "insufficient_data"
        
        # Simple trend calculation
        recent_performance = self.performance_history[-5:]
        
        # Calculate trend for each metric
        trends = []
        for metric_name in current_metrics:
            values = [record["metrics"].get(metric_name, 0) for record in recent_performance]
            if len(values) > 1:
                trend = "increasing" if values[-1] > values[0] else "decreasing"
                trends.append(f"{metric_name}:{trend}")
        
        return ", ".join(trends)

## 6. Example Usage and Demonstrations

In [ ]:
def demonstrate_langchain_agents():
    """Demonstrate LangChain agent capabilities"""
    
    print("=== LangChain Agent Framework Demo ===")
    
    # Initialize framework
    langchain_framework = LangChainAgentFramework(llm, tools)
    
    # Create various agents
    research_agent = langchain_framework.create_react_agent(
        "researcher",
        system_prompt="""You are a research specialist with access to search tools.
        Conduct thorough research and provide comprehensive summaries.
        Always verify information and cite sources."""
    )
    
    conversational_agent = langchain_framework.create_conversational_agent(
        "chatbot",
        system_prompt="""You are a friendly conversational AI assistant.
        Be engaging, helpful, and provide accurate information.
        Maintain context from previous messages."""
    )
    
    planning_agent = langchain_framework.create_planning_agent(
        "planner",
        system_prompt="""You are an expert planning assistant.
        Break down complex tasks into clear, manageable steps.
        Consider dependencies and provide logical sequences."""
    )
    
    print("\nAvailable agents:", langchain_framework.list_agents())
    
    # Demonstrate agent capabilities
    print("\n=== Testing Research Agent ===")
    research_result = langchain_framework.run_agent(
        "researcher",
        "Search for information about artificial intelligence trends in 2024"
    )
    print(f"Research result: {research_result[:200]}...")
    
    print("\n=== Testing Conversational Agent ===")
    conv_result = langchain_framework.run_agent(
        "chatbot",
        "Hello! I'm learning about AI agents. Can you tell me what makes them special?"
    )
    print(f"Conversation result: {conv_result[:200]}...")
    
    print("\n=== Testing Planning Agent ===")
    plan_result = langchain_framework.run_agent(
        "planner",
        "Plan the development of a simple AI chatbot application"
    )
    print(f"Planning result: {plan_result[:300]}...")
    
    return langchain_framework

def demonstrate_autogen_agents():
    """Demonstrate AutoGen agent capabilities"""
    
    print("\n=== AutoGen Agent Framework Demo ===")
    
    try:
        # Initialize framework
        autogen_framework = setup_autogen_example()
        
        print("\nAvailable agents:", autogen_framework.list_agents())
        print("Available group chats:", autogen_framework.list_group_chats())
        
        # Demonstrate group chat (this would require actual LLM calls)
        print("\n=== Group Chat Demonstration ===")
        print("Note: This would require actual LLM API calls to execute")
        print("The group chat would include: planner, coder, and reviewer agents")
        print("They would collaborate on a software development task")
        
        return autogen_framework
        
    except Exception as e:
        print(f"AutoGen demonstration skipped due to import issue: {e}")
        return None

def demonstrate_multi_agent_system():
    """Demonstrate multi-agent system with memory and learning"""
    
    print("\n=== Multi-Agent System Demo ===")
    
    # Initialize frameworks
    langchain_framework = LangChainAgentFramework(llm, tools)
    autogen_framework = setup_autogen_example()
    
    # Create multi-agent system
    multi_agent_system = MultiAgentSystem(langchain_framework, autogen_framework)
    
    # Create specialized agents
    agents = multi_agent_system.create_specialized_agents()
    
    # Create memory system
    memory_system = AgentMemorySystem()
    
    # Create learning agents
    learning_agents = {}
    for agent_name, agent in agents.items():
        learning_agents[agent_name] = LearningAgent(agent_name, memory_system)
    
    print(f"\nCreated {len(agents)} specialized agents:")
    for agent_name in agents.keys():
        print(f"  - {agent_name}")
    
    # Create workflow
    workflow_config = {
        "steps": [
            {
                "name": "research_step",
                "agent": "researcher",
                "task": "Research the topic: {topic}",
                "output_key": "research_results"
            },
            {
                "name": "analysis_step",
                "agent": "analyst",
                "task": "Analyze the research results: {research_results}",
                "output_key": "analysis_results"
            },
            {
                "name": "strategy_step",
                "agent": "strategist",
                "task": "Develop strategy based on: {analysis_results}",
                "output_key": "strategy"
            }
        ],
        "output_format": "detailed"
    }
    
    workflow = multi_agent_system.create_agent_workflow(
        "research_workflow",
        agents,
        workflow_config
    )
    
    print("\nCreated workflow: research_workflow")
    
    # Simulate workflow execution
    input_data = {"topic": "AI agent frameworks and their applications"}
    
    print(f"\nSimulating workflow execution with input: {input_data}")
    print("Note: This would require actual LLM calls to execute")
    
    # Demonstrate memory system
    print("\n=== Memory System Demo ===")
    
    # Add some test memories
    memory_system.add_semantic_memory(
        "agent_frameworks",
        {
            "description": "LangChain and AutoGen are popular AI agent frameworks",
            "langchain_features": ["tools", "chains", "agents"],
            "autogen_features": ["multi-agent", "conversation", "coding"]
        }
    )
    
    memory_system.add_episodic_memory({
        "event": "framework_comparison",
        "details": "Compared LangChain and AutoGen capabilities",
        "conclusion": "Both have strengths for different use cases"
    })
    
    # Test memory retrieval
    relevant_memories = memory_system.retrieve_relevant_memories(
        "What are the features of AI agent frameworks?",
        limit=2
    )
    
    print(f"Retrieved {len(relevant_memories)} relevant memories")
    for i, memory in enumerate(relevant_memories):
        print(f"Memory {i+1}: {memory.get('description', memory.get('event', 'Unknown'))}")
    
    # Show memory stats
    stats = memory_system.get_memory_stats()
    print(f"\nMemory System Stats: {stats}")
    
    return multi_agent_system, memory_system

def main_demonstration():
    """Main demonstration function"""
    
    print("🚀 AI Agent Frameworks Comprehensive Demonstration")
    print("=" * 60)
    
    # Demonstrate LangChain
    langchain_framework = demonstrate_langchain_agents()
    
    # Demonstrate AutoGen
    autogen_framework = demonstrate_autogen_agents()
    
    # Demonstrate multi-agent system
    multi_agent_system, memory_system = demonstrate_multi_agent_system()
    
    print("\n=== Summary ===")
    print("✓ LangChain agents with tool usage and memory")
    print("✓ AutoGen multi-agent conversation framework")
    print("✓ Advanced multi-agent workflows and coordination")
    print("✓ Memory systems with semantic and episodic storage")
    print("✓ Learning agents that adapt from interactions")
    
    print("\n🎯 Key Capabilities Demonstrated:")
    print("  • Tool-using agents with ReAct paradigm")
    print("  • Conversational agents with memory")
    print("  • Planning and task decomposition")
    print("  • Multi-agent collaboration")
    print("  • Workflow orchestration")
    print("  • Memory and learning systems")
    print("  • Agent coordination and communication")
    
    return {
        "langchain_framework": langchain_framework,
        "autogen_framework": autogen_framework,
        "multi_agent_system": multi_agent_system,
        "memory_system": memory_system
    }

# Run the demonstration
if __name__ == "__main__":
    frameworks = main_demonstration()

## 7. Performance Monitoring and Evaluation

In [ ]:
class AgentPerformanceMonitor:
    """Monitor and evaluate agent performance"""
    
    def __init__(self):
        self.metrics_history = []
        self.agent_stats = {}
        self.workflow_stats = {}
    
    def record_agent_execution(self, agent_name: str, execution_data: Dict):
        """Record agent execution metrics"""
        
        execution_record = {
            "timestamp": datetime.now(),
            "agent_name": agent_name,
            "execution_time": execution_data.get("execution_time", 0),
            "success": execution_data.get("success", False),
            "tokens_used": execution_data.get("tokens_used", 0),
            "tool_calls": execution_data.get("tool_calls", 0),
            "error_rate": execution_data.get("error_rate", 0),
            "user_satisfaction": execution_data.get("user_satisfaction", 0),
        }
        
        self.metrics_history.append(execution_record)
        
        # Update agent statistics
        if agent_name not in self.agent_stats:
            self.agent_stats[agent_name] = {
                "total_executions": 0,
                "total_time": 0,
                "successful_executions": 0,
                "total_tokens": 0,
                "total_tool_calls": 0,
            }
        
        stats = self.agent_stats[agent_name]
        stats["total_executions"] += 1
        stats["total_time"] += execution_record["execution_time"]
        if execution_record["success"]:
            stats["successful_executions"] += 1
        stats["total_tokens"] += execution_record["tokens_used"]
        stats["total_tool_calls"] += execution_record["tool_calls"]
    
    def record_workflow_execution(self, workflow_name: str, workflow_data: Dict):
        """Record workflow execution metrics"""
        
        workflow_record = {
            "timestamp": datetime.now(),
            "workflow_name": workflow_name,
            "total_time": workflow_data.get("total_time", 0),
            "steps_completed": workflow_data.get("steps_completed", 0),
            "steps_total": workflow_data.get("steps_total", 0),
            "success": workflow_data.get("success", False),
            "agents_involved": workflow_data.get("agents_involved", []),
        }
        
        self.metrics_history.append(workflow_record)
        
        # Update workflow statistics
        if workflow_name not in self.workflow_stats:
            self.workflow_stats[workflow_name] = {
                "total_executions": 0,
                "successful_executions": 0,
                "average_time": 0,
                "average_completion_rate": 0,
            }
        
        stats = self.workflow_stats[workflow_name]
        stats["total_executions"] += 1
        if workflow_record["success"]:
            stats["successful_executions"] += 1
        
        # Update averages
        executions = stats["total_executions"]
        stats["average_time"] = (stats["average_time"] * (executions - 1) + workflow_record["total_time"]) / executions
        
        completion_rate = workflow_record["steps_completed"] / max(workflow_record["steps_total"], 1)
        stats["average_completion_rate"] = (stats["average_completion_rate"] * (executions - 1) + completion_rate) / executions
    
    def get_agent_performance_report(self, agent_name: str) -> Dict:
        """Generate performance report for specific agent"""
        
        if agent_name not in self.agent_stats:
            return {"error": f"No data available for agent '{agent_name}'"}
        
        stats = self.agent_stats[agent_name]
        
        if stats["total_executions"] == 0:
            return {"error": "No executions recorded for this agent"}
        
        report = {
            "agent_name": agent_name,
            "total_executions": stats["total_executions"],
            "success_rate": stats["successful_executions"] / stats["total_executions"],
            "average_execution_time": stats["total_time"] / stats["total_executions"],
            "average_tokens_per_execution": stats["total_tokens"] / stats["total_executions"],
            "average_tool_calls": stats["total_tool_calls"] / stats["total_executions"],
            "efficiency_score": self._calculate_efficiency_score(agent_name),
        }
        
        return report
    
    def get_workflow_performance_report(self, workflow_name: str) -> Dict:
        """Generate performance report for specific workflow"""
        
        if workflow_name not in self.workflow_stats:
            return {"error": f"No data available for workflow '{workflow_name}'"}
        
        stats = self.workflow_stats[workflow_name]
        
        report = {
            "workflow_name": workflow_name,
            "total_executions": stats["total_executions"],
            "success_rate": stats["successful_executions"] / stats["total_executions"],
            "average_execution_time": stats["average_time"],
            "average_completion_rate": stats["average_completion_rate"],
            "reliability_score": self._calculate_reliability_score(workflow_name),
        }
        
        return report
    
    def get_system_overview(self) -> Dict:
        """Get system-wide performance overview"""
        
        total_executions = len(self.metrics_history)
        recent_executions = [record for record in self.metrics_history 
                            if (datetime.now() - record["timestamp"]).days < 7]
        
        overview = {
            "total_executions": total_executions,
            "recent_executions_7d": len(recent_executions),
            "agents_monitored": len(self.agent_stats),
            "workflows_monitored": len(self.workflow_stats),
            "overall_success_rate": self._calculate_overall_success_rate(),
            "average_execution_time": self._calculate_average_execution_time(),
            "system_health_score": self._calculate_system_health_score(),
        }
        
        return overview
    
    def _calculate_efficiency_score(self, agent_name: str) -> float:
        """Calculate efficiency score for agent"""
        if agent_name not in self.agent_stats:
            return 0.0
        
        stats = self.agent_stats[agent_name]
        
        if stats["total_executions"] == 0:
            return 0.0
        
        # Efficiency = success_rate / (execution_time + 1) * (1 + tool_calls / executions)
        success_rate = stats["successful_executions"] / stats["total_executions"]
        avg_time = stats["total_time"] / stats["total_executions"]
        avg_tool_calls = stats["total_tool_calls"] / stats["total_executions"]
        
        efficiency = success_rate / (avg_time + 1) * (1 + avg_tool_calls)
        return min(efficiency * 100, 100)  # Scale to 0-100
    
    def _calculate_reliability_score(self, workflow_name: str) -> float:
        """Calculate reliability score for workflow"""
        if workflow_name not in self.workflow_stats:
            return 0.0
        
        stats = self.workflow_stats[workflow_name]
        
        # Reliability = success_rate * completion_rate
        success_rate = stats["successful_executions"] / stats["total_executions"]
        completion_rate = stats["average_completion_rate"]
        
        return (success_rate * completion_rate) * 100
    
    def _calculate_overall_success_rate(self) -> float:
        """Calculate overall system success rate"""
        if not self.metrics_history:
            return 0.0
        
        successful = sum(1 for record in self.metrics_history if record.get("success", False))
        return (successful / len(self.metrics_history)) * 100
    
    def _calculate_average_execution_time(self) -> float:
        """Calculate average execution time"""
        if not self.metrics_history:
            return 0.0
        
        execution_times = [record.get("execution_time", 0) for record in self.metrics_history]
        return sum(execution_times) / len(execution_times)
    
    def _calculate_system_health_score(self) -> float:
        """Calculate overall system health score"""
        # Health = (success_rate * 0.4) + (efficiency * 0.3) + (reliability * 0.3)
        success_rate = self._calculate_overall_success_rate()
        
        # Calculate average efficiency
        if self.agent_stats:
            avg_efficiency = sum(self._calculate_efficiency_score(agent) for agent in self.agent_stats) / len(self.agent_stats)
        else:
            avg_efficiency = 0
        
        # Calculate average reliability
        if self.workflow_stats:
            avg_reliability = sum(self._calculate_reliability_score(workflow) for workflow in self.workflow_stats) / len(self.workflow_stats)
        else:
            avg_reliability = 0
        
        health_score = (success_rate * 0.4) + (avg_efficiency * 0.3) + (avg_reliability * 0.3)
        return min(health_score, 100)
    
    def export_metrics(self, filename: str = "agent_metrics.json"):
        """Export metrics to JSON file"""
        export_data = {
            "export_timestamp": datetime.now().isoformat(),
            "agent_stats": self.agent_stats,
            "workflow_stats": self.workflow_stats,
            "system_overview": self.get_system_overview(),
            "recent_metrics": self.metrics_history[-100:]  # Last 100 records
        }
        
        with open(filename, 'w') as f:
            json.dump(export_data, f, indent=2, default=str)
        
        print(f"Metrics exported to {filename}")

## 8. Summary and Best Practices

### Key Capabilities Demonstrated:

1. **LangChain Agents**:
   - ReAct agents with tool usage
   - Conversational agents with memory
   - Planning and task decomposition
   - Multi-tool coordination
   - Custom tool creation

2. **AutoGen Framework**:
   - Multi-agent conversation systems
   - Group chat management
   - Specialized agent roles
   - Asynchronous agent communication

3. **Multi-Agent Systems**:
   - Workflow orchestration
   - Agent coordination
   - Dependency management
   - Parallel execution

4. **Memory and Learning**:
   - Short-term and long-term memory
   - Semantic and episodic memory
   - Memory retrieval and indexing
   - Learning from interactions
   - Behavior adaptation

5. **Performance Monitoring**:
   - Agent performance metrics
   - Workflow execution tracking
   - Efficiency and reliability scoring
   - System health monitoring

### Best Practices:

1. **Agent Design**:
   - Define clear roles and responsibilities
   - Use appropriate system prompts
   - Implement proper error handling
   - Include memory for context retention

2. **Tool Usage**:
   - Provide clear tool descriptions
   - Implement proper error handling
   - Use tools judiciously
   - Validate tool inputs and outputs

3. **Multi-Agent Coordination**:
   - Define clear communication protocols
   - Manage agent dependencies
   - Implement proper workflow orchestration
   - Handle conflicts and failures gracefully

4. **Memory Management**:
   - Balance memory usage and performance
   - Implement efficient retrieval mechanisms
   - Use appropriate memory consolidation strategies
   - Respect privacy and data retention policies

5. **Performance Optimization**:
   - Monitor key performance metrics
   - Optimize agent response times
   - Balance between capability and efficiency
   - Implement proper caching strategies

### Next Steps:

- Implement actual LLM API integration
- Add more sophisticated tool implementations
- Implement agent learning and adaptation algorithms
- Add visualization for agent workflows
- Implement agent security and access control
- Add agent versioning and deployment management